In [7]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import plotly.express as px

In [11]:
btc_macro_df = pd.read_csv('btc_macroeconomic.csv')
print(btc_macro_df.shape)
btc_macro_df.head()

(3651, 12)


,date,btc_price_usd,supply_btc,btc_market_cap_usd,gold_price_usd,SP500,US_1_year_treasury_yield,US_10_year_treasury_yield,fed_funds_rate,US_M2_money_supply_in_billions,US_inflation,Nasdaq_100
0,30/3/2015,246,14000290,3.438726e+09,1187,2086,0.27,1.96,0.12,11904,-0.10,4383
1,31/3/2015,246,14004040,3.440500e+09,1179,2068,0.26,1.94,0.06,11904,-0.10,4334
2,1/4/2015,244,14007340,3.416815e+09,1181,2060,0.27,1.87,0.12,11938,-0.23,4311
3,2/4/2015,249,14011490,3.484769e+09,1202,2067,0.25,1.92,0.12,11938,-0.23,4316
4,3/4/2015,254,14015140,3.557376e+09,No data,No data,0.21,1.85,0.12,11938,-0.23,No data


In [15]:
btc_macro_df.describe(include='all')

,date,btc_price_usd,supply_btc,btc_market_cap_usd,gold_price_usd,SP500,US_1_year_treasury_yield,US_10_year_treasury_yield,fed_funds_rate,US_M2_money_supply_in_billions,US_inflation,Nasdaq_100
count,3651,3651.000000,3.651000e+03,3.651000e+03,3651,3651,3651,3651,3651,3651.000000,3651.000000,3651
unique,3651,NaN,NaN,NaN,1001,1697,458,416,93,NaN,NaN,2241
top,30/3/2015,NaN,NaN,NaN,No data,No data,No data,No data,No data,NaN,NaN,No data
freq,1,NaN,NaN,NaN,1080,1136,1152,1150,782,NaN,NaN,1135
mean,NaN,21007.209532,1.781319e+07,4.008236e+11,NaN,NaN,NaN,NaN,NaN,17151.669406,2.888398,NaN
std,NaN,22849.347813,1.604355e+06,4.483051e+11,NaN,NaN,NaN,NaN,NaN,3637.522872,2.225780,NaN
min,NaN,215.000000,1.400029e+07,3.089307e+09,NaN,NaN,NaN,NaN,NaN,11904.000000,-0.230000,NaN
25%,NaN,3644.000000,1.659180e+07,6.352240e+10,NaN,NaN,NaN,NaN,NaN,13723.000000,1.580000,NaN
50%,NaN,10360.000000,1.829222e+07,1.851300e+11,NaN,NaN,NaN,NaN,NaN,15994.000000,2.240000,NaN
75%,NaN,36405.500000,1.916199e+07,6.908845e+11,NaN,NaN,NaN,NaN,NaN,20902.000000,3.210000,NaN


In [3]:
#np.nan used to stop errors in mathematical operations.
btc_macro_df = btc_macro_df.replace('No data', np.nan)
btc_macro_df


,date,btc_price_usd,supply_btc,btc_market_cap_usd,gold_price_usd,SP500,US_1_year_treasury_yield,US_10_year_treasury_yield,fed_funds_rate,US_M2_money_supply_in_billions,US_inflation,Nasdaq_100
0,30/3/2015,246,14000290,3.438726e+09,1187,2086,0.27,1.96,0.12,11904,-0.10,4383
1,31/3/2015,246,14004040,3.440500e+09,1179,2068,0.26,1.94,0.06,11904,-0.10,4334
2,1/4/2015,244,14007340,3.416815e+09,1181,2060,0.27,1.87,0.12,11938,-0.23,4311
3,2/4/2015,249,14011490,3.484769e+09,1202,2067,0.25,1.92,0.12,11938,-0.23,4316
4,3/4/2015,254,14015140,3.557376e+09,NaN,NaN,0.21,1.85,0.12,11938,-0.23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3646,23/3/2025,84788,19841040,1.682280e+12,NaN,NaN,NaN,NaN,NaN,21671,2.75,NaN
3647,24/3/2025,87400,19841521,1.734150e+12,3010,5768,4.11,4.34,4.33,21671,2.75,20180
3648,25/3/2025,87371,19841987,1.733620e+12,3021,5777,4.09,4.31,4.33,21671,2.75,20288
3649,26/3/2025,87382,19842462,1.733870e+12,3019,5712,NaN,4.35,4.33,21671,2.75,19917


In [4]:
btc_macro_df['date'] = pd.to_datetime(btc_macro_df['date'], format='%d/%m/%Y')


In [5]:
feature_cols = ['gold_price_usd', 'SP500', 'fed_funds_rate', 'US_inflation', 'US_M2_money_supply_in_billions']

X = btc_macro_df[feature_cols]
y = btc_macro_df['btc_price_usd']

total_rows = len(X)
na_rows = X.isna().any(axis=1).sum()
print(f"Total rows: {total_rows}")
print(f"Rows with at least one NaN: {na_rows} ({na_rows/total_rows:.2%})")

mask = ~X.isna().any(axis=1)

X_clean = X[mask]
y_clean = y[mask]

print(f"Rows after dropping NaNs: {len(X_clean)}")

Total rows: 3651
Rows with at least one NaN: 1166 (31.94%)
Rows after dropping NaNs: 2485


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, random_state=123)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)



In [19]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.linear_model import LinearRegression

# Function to generate sample data
def generate_data():
    np.random.seed(42)
    gold_prices = np.random.uniform(1000, 2500, 100)
    btc_prices = 10000 + 15 * gold_prices + np.random.normal(0, 2000, 100)
    return pd.DataFrame({
        'gold_price_usd': gold_prices,
        'btc_price_usd': btc_prices
    })

# Function to train model
def train_model():
    df = generate_data()
    X = df[['gold_price_usd']]
    y = df['btc_price_usd']
    model = LinearRegression()
    model.fit(X, y)
    return model

def main():
    st.title('BTC price predictor against Macro conditions')
    st.write("Macroeconomics affect BTC's price")  # Fixed apostrophe issue
    
    # Train model
    model = train_model()
    
    # User input
    gold_price = st.number_input('Gold Price (USD)', 
                          min_value=0, 
                          max_value=4000, 
                          value=1500)
    
    if st.button('Predict price'):
        # Perform prediction
        prediction = model.predict([[gold_price]])
        
        # Show result
        st.success(f'Estimated BTC price: ${prediction[0]:,.2f}')
        
        # Visualization
        df = generate_data()
        fig = px.scatter(df, x='gold_price_usd', y='btc_price_usd', 
                       title='Gold Price vs BTC Price Relationship')
        
        # Add prediction point
        import plotly.graph_objects as go
        fig.add_trace(
            go.Scatter(
                x=[gold_price], 
                y=[prediction[0]], 
                mode='markers',
                marker=dict(size=15, color='red'),
                name='Prediction'
            )
        )
        
        st.plotly_chart(fig)
 
if __name__ == '__main__':
    main()

2025-04-13 16:27:11.752 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 16:27:11.753 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 16:27:11.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 16:27:11.757 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 16:27:11.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 16:27:11.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 16:27:11.778 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-13 16:27:11.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar